<a href="https://colab.research.google.com/github/firdausmulla/AI-Style-Mixed/blob/main/AI_Artist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# Cell 1: Install All Libraries
!pip install flask pyngrok tensorflow tensorflow_hub

In [49]:
# Cell 2: Authenticate ngrok
from pyngrok import ngrok
import getpass

# Get your authtoken from the ngrok dashboard
authtoken = getpass.getpass("Paste your ngrok authtoken here: ")
ngrok.set_auth_token(authtoken)

Paste your ngrok authtoken here: ··········


In [50]:
# Cell 3: Create the HTML Webpage

!mkdir -p templates
print("Creating new index.html file...")

Creating new index.html file...


In [51]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Style Mixer</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif; background-color: #f4f7f6; color: #333; display: flex; justify-content: center; align-items: flex-start; padding: 20px; min-height: 100vh; }
        .container { width: 100%; max-width: 1000px; background-color: #ffffff; padding: 30px; border-radius: 12px; box-shadow: 0 8px 30px rgba(0,0,0,0.05); text-align: center; }
        h1 { color: #2c3e50; margin-bottom: 10px; }
        p { color: #7f8c8d; margin-bottom: 25px; }
        form { margin-bottom: 30px; border: 1px solid #eee; padding: 25px; border-radius: 10px; background-color: #fdfdfd; }

        .upload-section {
            display: flex;
            justify-content: space-around;
            gap: 20px;
            margin-bottom: 20px;
        }
        .upload-box {
            flex: 1;
            padding: 15px;
            border: 2px dashed #3498db;
            border-radius: 8px;
        }
        .upload-box label { font-weight: bold; color: #34495e; display: block; margin-bottom: 10px; }

        input[type="file"] { width: 100%; }
        input[type="submit"] { display: block; width: 100%; padding: 15px; font-size: 18px; font-weight: bold; color: #fff; background-color: #3498db; border: none; border-radius: 8px; cursor: pointer; transition: background-color 0.3s ease; margin-top: 20px; }
        input[type="submit"]:hover { background-color: #2980b9; }
        #loader { display: none; margin-top: 20px; }
        #loader p { font-size: 18px; color: #7f8c8d; }

        .image-results { display: flex; flex-wrap: wrap; justify-content: center; gap: 20px; margin-top: 30px; }
        .image-box { flex: 1; min-width: 250px; max-width: 32%; text-align: center; background-color: #fafafa; padding: 15px; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.05); }
        .image-box h3 { color: #34495e; margin-bottom: 10px; }
        .image-box img { max-width: 100%; height: auto; border-radius: 6px; border: 1px solid #ddd; }

        @media (max-width: 768px) {
            .upload-section { flex-direction: column; }
            .image-box { max-width: 100%; }
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>AI Style Mixer 🎨</h1>
        <p>Upload a **Content** image and a **Style** image to create a masterpiece!</p>

        <form action="/stylize" method="POST" enctype="multipart/form-data" onsubmit="showLoader()">
            <div class="upload-section">
                <div class="upload-box">
                    <label for="content_image">1. Upload Your Content Image</label>
                    <input type="file" name="content_image" id="content_image" accept="image/*" required>
                </div>
                <div class="upload-box">
                    <label for="style_image">2. Upload Your Style Image</label>
                    <input type="file" name="style_image" id="style_image" accept="image/*" required>
                </div>
            </div>
            <input type="submit" value="Mix the Styles!">
        </form>

        <div id="loader">
            <p>🖌️ Please wait... The AI is painting your image...</p>
        </div>

        {% if content_image_b64 and style_image_b64 and stylized_image_b64 %}
        <div class="image-results">
            <div class="image-box">
                <h3>1. Your Content</h3>
                <img src="data:image/jpeg;base64,{{ content_image_b64 }}" alt="Original Image">
            </div>
            <div class="image-box">
                <h3>2. Your Style</h3>
                <img src="data:image/jpeg;base64,{{ style_image_b64 }}" alt="Style Image">
            </div>
            <div class="image-box">
                <h3>3. Your AI Art</h3>
                <img src="data:image/jpeg;base64,{{ stylized_image_b64 }}" alt="Stylized Image">
            </div>
        </div>
        {% endif %}
    </div>

    <script>
        function showLoader() {
            document.getElementById('loader').style.display = 'block';
            document.querySelector('input[type="submit"]').disabled = true;
            document.querySelector('input[type="submit"]').value = 'Processing...';
        }
        window.onload = function() {
            if (document.querySelector('.image-results')) {
                document.getElementById('loader').style.display = 'none';
            }
        };
    </script>
</body>
</html>

Overwriting templates/index.html


In [52]:
# Cell 4 (FINAL): Run the "AI Style Mixer" App

import os
import tensorflow as tf

# --- THIS IS THE FIX ---
# We create a new, clean cache directory and force TF_HUB to use it.
# This guarantees the *one* model we need will download cleanly.
new_cache_dir = '/content/tf_cache'
!mkdir -p {new_cache_dir}
os.environ['TFHUB_CACHE_DIR'] = new_cache_dir
print(f"Forcing TensorFlow Hub to use new, clean cache: {new_cache_dir}")
# --- END OF FIX ---

from flask import Flask, request, render_template
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import threading
import io
import base64
from pyngrok import ngrok # Import ngrok here

# --- 1. Helper Functions for Image Processing ---

def load_img_from_stream(stream):
    """Loads an image from an UPLOAD STREAM and prepares it."""
    max_dim = 512
    img = Image.open(stream).convert('RGB')
    img = np.array(img)
    img = tf.convert_to_tensor(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def tensor_to_base64_string(tensor):
    """Converts the output image tensor (range [0,1]) to a base64 string."""
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        tensor = tensor[0]

    pil_img = Image.fromarray(tensor)
    buff = io.BytesIO()
    pil_img.save(buff, format="JPEG")
    base64_string = base64.b64encode(buff.getvalue()).decode("utf-8")
    return base64_string

# --- 2. Load the ONE Model We Need ---
print("Loading ML Style Transfer model...")
# This will be a fresh, clean download to our new cache dir.
style_transfer_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
print("Style Transfer model loaded.")

# --- 3. Initialize the Flask App ---
app = Flask(__name__)

@app.route('/')
def home():
    """Serves the main page."""
    return render_template('index.html')

@app.route('/stylize', methods=['POST'])
def stylize_image():
    """Handles the image upload, runs the model, and shows the result."""
    try:
        # --- Part 1: Load User's Images ---
        content_file = request.files['content_image']
        style_file = request.files['style_image']

        content_image_tensor = load_img_from_stream(content_file.stream) # Range [0,1]
        style_image_tensor = load_img_from_stream(style_file.stream) # Range [0,1]

        # --- Part 2: ML Applies the Style ---
        print("Applying user-provided style to content image...")
        stylized_image_tensor = style_transfer_model(
            tf.constant(content_image_tensor),
            tf.constant(style_image_tensor)
        )[0] # Output is [0,1]
        print("Stylization complete.")

        # --- Part 3: Convert all 3 images for display ---
        content_image_b64 = tensor_to_base64_string(content_image_tensor)
        style_image_b64 = tensor_to_base64_string(style_image_tensor)
        stylized_image_b64 = tensor_to_base64_string(stylized_image_tensor)

        return render_template('index.html',
                               content_image_b64=content_image_b64,
                               style_image_b64=style_image_b64,
                               stylized_image_b64=stylized_image_b64)

    except Exception as e:
        print(f"Error during stylization: {e}")
        return f"An error occurred: {e}. Please try again.", 500

# --- 4. Start the App and Tunnel ---
def run_flask_app():
    print("Starting Flask app...")
    app.run(port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()

print("Connecting to ngrok...")
try:
    ngrok.kill()
except Exception:
    pass
public_url = ngrok.connect(5000)
print(f" * Your AI Artist app is live at: {public_url}")
print(" * (Click the URL above to open your app. This cell will keep running.)")

Forcing TensorFlow Hub to use new, clean cache: /content/tf_cache
Loading ML Style Transfer model...
Style Transfer model loaded.
Starting Flask app...
Connecting to ngrok...
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


 * Your AI Artist app is live at: NgrokTunnel: "https://unstraddled-katia-eyelike.ngrok-free.dev" -> "http://localhost:5000"
 * (Click the URL above to open your app. This cell will keep running.)
